In [1]:
include("../src/NursesScheduling.jl")
using .NurseSchedules
using Logging

logger = ConsoleLogger(stderr, Logging.Debug)

ConsoleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(0x00000030) open, 0 bytes waiting))), Debug, Logging.default_metafmt, true, 0, Dict{Any,Int64}())

In [2]:
nurse_schedule = Schedule("schedules/schedule_2016_august_medium.json")

┌ Info: Schedule loaded!
└ @ Main.NurseSchedules /home/prenc/Documents/nurse/nurse_scheduling_problem/src/Schedule.jl:10


Schedule(Dict{String,Any}("shifts" => Dict{String,Any}("nurse_2" => Any["N", "W", "N", "W", "W", "W", "DN"],"babysitter_1" => Any["U", "U", "U", "U", "U", "U", "U"],"babysitter_7" => Any["W", "D", "D", "W", "D", "DN", "W"],"nurse_5" => Any["W", "W", "W", "W", "W", "W", "W"],"babysitter_8" => Any["L4", "L4", "L4", "L4", "L4", "L4", "L4"],"babysitter_9" => Any["N", "W", "D", "N", "P", "W", "W"],"nurse_1" => Any["R", "R", "W", "W", "D", "DN", "W"],"babysitter_10" => Any["W", "D", "W", "D", "W", "D", "W"],"nurse_6" => Any["U", "U", "U", "U", "U", "U", "U"],"babysitter_5" => Any["D", "P", "N", "N", "W", "W", "W"]…),"schedule_info" => Dict{String,Any}("UUID" => "","month_number" => 8,"year" => 2016),"month_info" => Dict{String,Any}("children_number" => Any[12, 12, 12, 12, 12, 12, 11],"days_of_week" => Any["MO", "TU", "WE", "TH", "FR", "SA", "SU"],"day_numbers" => Any[8, 9, 10, 11, 12, 13, 14]),"employee_info" => Dict{String,Any}("time" => Dict{String,Any}("nurse_2" => "FULL","babysitter_1" =

In [3]:
schedule_shifts = get_shifts(nurse_schedule)
workers, shifts = schedule_shifts
month_info = get_month_info(nurse_schedule)
workers_info = get_workers_info(nurse_schedule)

Dict{String,Any} with 2 entries:
  "time" => Dict{String,Any}("nurse_2"=>"FULL","babysitter_1"=>"FULL","babysitt…
  "type" => Dict{String,Any}("nurse_2"=>"NURSE","babysitter_1"=>"OTHER","babysi…

In [4]:
with_logger(logger) do
    get_max_nbhd_size(nurse_schedule)
end

┌ Debug: Neighbors number from addition: 252
└ @ Main.NurseSchedules.NeighborhoodGen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:15
┌ Debug: Neighbors number from deletion: 49
└ @ Main.NurseSchedules.NeighborhoodGen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:17
┌ Debug: Neighbors number from swap: 116
└ @ Main.NurseSchedules.NeighborhoodGen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:29


417

In [5]:
nbhd = Neighborhood(shifts)
length(nbhd)

417

In [6]:
with_logger(logger) do
    global schedule_score = score(schedule_shifts, month_info, workers_info)
end

┌ Debug: There is a lack of staff on day '7'.
│ Expected '4', got '3' in the day.
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:127
┌ Debug: Lacking workers total penalty: 30
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:68
┌ Debug: The worker 'babysitter_7' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:215
┌ Debug: The worker 'babysitter_9' has a disallowed shift sequence on day '5': N -> P
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:188
┌ Debug: The worker 'babysitter_10' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:215
┌ Debug: The worker 'nurse_8' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/n

(penalty = 216, errors = Dict{String,Any}[Dict("day" => 7,"required" => 4,"code" => "WND","actual" => 3), Dict("worker" => "babysitter_7","week" => 1,"code" => "LLB"), Dict("day" => 5,"succeeding" => "P","worker" => "babysitter_9","code" => "DSS","preceding" => "N"), Dict("worker" => "babysitter_10","week" => 1,"code" => "LLB"), Dict("worker" => "nurse_8","week" => 1,"code" => "LLB"), Dict("worker" => "nurse_4","week" => 1,"code" => "LLB"), Dict("day" => 4,"succeeding" => "N","worker" => "nurse_3","code" => "DSS","preceding" => "DN"), Dict("day" => 7,"succeeding" => "N","worker" => "nurse_3","code" => "DSS","preceding" => "DN"), Dict("worker" => "nurse_3","week" => 1,"code" => "LLB"), Dict("worker" => "babysitter_4","week" => 1,"code" => "LLB"), Dict("hours" => 20,"worker" => "nurse_5","code" => "WUH"), Dict("hours" => 16,"worker" => "nurse_3","code" => "WOH")])

In [7]:
ITERATION_NUM = 1000
best_shifts = (shifts, schedule_score)

(["N" "W" … "W" "DN"; "U" "U" … "U" "U"; … ; "U" "U" … "U" "U"; "D" "N" … "W" "DN"], (penalty = 216, errors = Dict{String,Any}[Dict("day" => 7,"required" => 4,"code" => "WND","actual" => 3), Dict("worker" => "babysitter_7","week" => 1,"code" => "LLB"), Dict("day" => 5,"succeeding" => "P","worker" => "babysitter_9","code" => "DSS","preceding" => "N"), Dict("worker" => "babysitter_10","week" => 1,"code" => "LLB"), Dict("worker" => "nurse_8","week" => 1,"code" => "LLB"), Dict("worker" => "nurse_4","week" => 1,"code" => "LLB"), Dict("day" => 4,"succeeding" => "N","worker" => "nurse_3","code" => "DSS","preceding" => "DN"), Dict("day" => 7,"succeeding" => "N","worker" => "nurse_3","code" => "DSS","preceding" => "DN"), Dict("worker" => "nurse_3","week" => 1,"code" => "LLB"), Dict("worker" => "babysitter_4","week" => 1,"code" => "LLB"), Dict("hours" => 20,"worker" => "nurse_5","code" => "WUH"), Dict("hours" => 16,"worker" => "nurse_3","code" => "WOH")]))

In [ ]:
iter_best = best_shifts
no_improvement_iters = 0

for i in 1:ITERATION_NUM
    nbhd = Neighborhood(iter_best[1])
    nghd_scores = map(shifts -> score((workers, shifts), month_info, workers_info), nbhd)
    show(nghd_scores)
    iter_best_idx = findfirst(nghd_scores .== minimum(nghd_scores))
    iter_best = (nbhd[iter_best_idx], nghd_scores[iter_best_idx])
    
    no_improvement_iters += 1
    
    if best_shifts[2] > iter_best[2]
        println("Improved from $(best_shifts[2]) to $(iter_best[2]) in iteration '$(i)'")
        global no_improvement_iters = 0
        global best_shifts = iter_best
    end
    if no_improvement_iters > 20
        println("Local minimum! Stopping in iteration '$(i)'")
        break
    end
end

In [ ]:
with_logger(logger) do
    penalty, _ = score((workers, best_shifts[1]), month_info, workers_info)
end